In [1]:
import os
from langchain_community.document_loaders import TextLoader

In [3]:
files_folder = "./movie_text_data/"
files = os.listdir(files_folder)

docs = []

for file in files:
    loader = TextLoader(files_folder + file)
    docs.extend(loader.load())
    
len(docs)

840

In [4]:
from langchain_community.embeddings import OllamaEmbeddings

embeddings_llm = OllamaEmbeddings(model="llama3") #base_url = 'http://localhost:11434'

In [5]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter()

documents = text_splitter.split_documents(docs)

len(documents)

840

In [6]:
from langchain_community.vectorstores import FAISS

vector_index = FAISS.from_documents(documents, embeddings_llm)

vector_index

### There are 3 functions to create index.

from_documents()
from_embeddings()
from_texts()

In [12]:
retriever = vector_index.as_retriever()

relevant_docs = retriever.invoke({"input": "Tales from the Hood"})

len(relevant_docs)

4

In [13]:
relevant_docs

[Document(page_content="Title: What's Eating Gilbert Grape\nDirector: Lasse Hallström\nGenres: Drama\nBrief Plot: A young man in a small Midwestern town struggles to care for his mentally-disabled younger brother and morbidly obese mother while attempting to pursue his own happiness.\nYear: 1993\nStars: Johnny Depp, Leonardo DiCaprio, Juliette Lewis", metadata={'source': './movie_text_data/movie_tt0108550.txt'}),
 Document(page_content='Title: Casino\nDirector: Martin Scorsese\nGenres: Crime, Drama\nBrief Plot: In Las Vegas, two best friends - a casino executive and a mafia enforcer - compete for a gambling empire and a fast-living, fast-loving socialite.\nYear: 1995\nStars: Robert De Niro, Sharon Stone, Joe Pesci', metadata={'source': './movie_text_data/movie_tt0112641.txt'}),
 Document(page_content='Title: Ace Ventura: Pet Detective\nDirector: Tom Shadyac\nGenres: Comedy\nBrief Plot: A goofy detective specializing in animals goes in search of the missing mascot of the Miami Dolphins.

In [9]:
for doc in relevant_docs:
    print(f"Title : {doc.metadata['title']}, Source: {doc.metadata['source']}")

KeyError: 'title'

In [27]:
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.documents import Document

prompt = ChatPromptTemplate.from_template("""
Answer the following question based on the provided context and your internal knowledge.
Give priority to context and if you are not sure then say you are not aware of topic.
Stop your generation when you end the concept: never generate more than 32 tokens.

<context>
{context}
</context>

Question: {input}
""")

#document_chain  = prompt | llm 
document_chain = create_stuff_documents_chain(llm, prompt)

response = document_chain.invoke({
    "input": "Do you know about Claude 3?",
    "context": [Document(page_content="Claude 3 is latest Conversational AI Model from Anthropic.")]
})

print(response)

Yes, I am familiar with Claude 3. It's a conversational AI model from Anthropic, the latest one in their series of Claude models.<|eot_id|><|start_header_id|>assistant<|end_header_id|>

(32 tokens)<|eot_id|><|start_header_id|>assistant<|end_header_id|>

I'll stop here since I reached the token limit!<|eot_id|><|start_header_id|>assistant<|end_header_id|>

It seems I didn't quite reach the 32-token limit after all!<|eot_id|><|start_header_id|>assistant<|end_header_id|>

Here's my answer again, within the 32-token limit:

Yes, I know about Claude 3. It's a conversational AI model from Anthropic.<|eot_id|><|start_header_id|>assistant<|end_header_id|>

I think I got it right this time!<|eot_id|><|start_header_id|>assistant<|end_header_id|>

( Done!)<|eot_id|><|start_header_id|>assistant<|end_header_id|>

I'll stop here since I reached the token limit!<|eot_id|><|start_header_id|>assistant<|end_header_id|>

(32 tokens)<|eot_id|><|start_header_id|>assistant<|end_header_id|>

Let me try again

In [18]:
from langchain.chains import create_retrieval_chain

retrieval_chain = create_retrieval_chain(retriever, document_chain)

retrieval_chain

RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableBinding(bound=RunnableLambda(lambda x: x['input'])
           | VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x109aa6990>), config={'run_name': 'retrieve_documents'})
})
| RunnableAssign(mapper={
    answer: RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
              context: RunnableLambda(format_docs)
            }), config={'run_name': 'format_inputs'})
            | ChatPromptTemplate(input_variables=['context', 'input'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'input'], template='\nAnswer the following question based on the provided context and your internal knowledge.\nGive priority to context and if you are not sure then say you are not aware of topic:\n\n<context>\n{context}\n</context>\n\nQuestion: {input}\n'))])
            | Ollama(model='llama3')
            | Str

In [19]:
response = retrieval_chain.invoke({"input": "Do you know about Claude 3?"})

type(response)

dict

In [20]:
print(response["answer"])

Claude 3 is a powerful language model developed by Anthropic, a leading AI research organization. It's designed to be highly intelligent and capable of understanding complex tasks, with three variants: Opus, Sonnet, and Haiku.

Opus is the most intelligent model, ideal for complex analysis, longer tasks with multiple steps, and higher-order math and coding tasks.

Sonnet is engineered for high endurance in large-scale AI deployments, offering a balance between intelligence and speed at a lower cost compared to its peers.

Haiku is the fastest and most compact model, perfect for near-instant responsiveness and answering simple queries and requests quickly.

Each variant has its unique strengths and use cases, making Claude 3 an excellent choice for various applications, from coding assistance to data processing and content moderation.

What would you like to know more about Claude 3 or how it can be applied in a specific context?


In [21]:
for doc in response["context"]:
    print(f"Title : {doc.metadata['title']}, Source: {doc.metadata['source']}")

Title : Introducing the next generation of Claude \ Anthropic, Source: https://www.anthropic.com/news/claude-3-family
Title : Claude \ Anthropic, Source: https://www.anthropic.com/claude
Title : Claude 2 \ Anthropic, Source: https://www.anthropic.com/news/claude-2
Title : Introducing the next generation of Claude \ Anthropic, Source: https://www.anthropic.com/news/claude-3-family
